<a href="https://colab.research.google.com/github/llA-Wilsonll/Simpsons_Character_Recognition/blob/master/The_Simpsons_Character_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading the Data

Importing the required packages

In [0]:
import os
import zipfile
import sys
import shutil
import random
import re

Upload the kaggle json Key which allows us to download data from Kaggle.

In [3]:
from google.colab import files
files.upload()  #this will prompt you to upload the kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aaroncwilson","key":"8fd509bef31a87eec6f17681d818cf03"}'}

The following packages are used so we can save files from Colab to Google Drive and vice versa.

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Installing a package that will allow us to search for images.

In [11]:
pip install google_images_download

     |████████████████████████████████| 911kB 4.2MB/s 
  Created wheel for google-images-download: filename=google_images_download-2.8.0-py2.py3-none-any.whl size=14550 sha256=7d33940ffc192a63cc6a3e30dc70a262f9a495834f07ed3e965c81b229dcbcae
  Stored in directory: /root/.cache/pip/wheels/1f/28/ad/f56e7061e1d2a9a1affe2f9c649c2570cb9198dd24ede0bbab
Successfully built google-images-download


In [0]:
from google_images_download import google_images_download   #importing the library

## Downloading the Kaggle Dataset
We start by downloading the dataset prepared by alexattia on Kaggle: https://www.kaggle.com/alexattia/the-simpsons-characters-dataset

In [5]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission

kaggle.json


In [6]:
!kaggle datasets download -d alexattia/the-simpsons-characters-dataset -p /content/

100% 1.07G/1.08G [00:34<00:00, 33.6MB/s]
100% 1.08G/1.08G [00:34<00:00, 33.7MB/s]


We then create the directory where we want to store the data and then unzip the file we downloaded into it.

In [0]:
os.mkdir('/content/simpsons/')

In [0]:
local_zip = '/content/the-simpsons-characters-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/simpsons')
zip_ref.close()

To get an idea of how many images we have of each character we run the below code.

In [9]:
characters = os.listdir('/content/simpsons/simpsons_dataset/simpsons_dataset')
print('There are ' + str(len(characters)) + ' characters.')

# Creating a dictionary to store the number of images per character
character_observations_dict = {}
for character in characters:
  num_images = len(os.listdir('/content/simpsons/simpsons_dataset/simpsons_dataset/' 
                              + character))
  character_observations_dict[character] = num_images
  
listofTuples = sorted(character_observations_dict.items(), reverse=True,
                      key=lambda x: x[1])
 
# Iterate over the sorted sequence
for elem in listofTuples :
    print("{0:<26}{1:^2}{2:>13}".format(elem[0],"::", str(elem[1]) + " images."))

There are 42 characters.
homer_simpson             :: 2246 images.
ned_flanders              :: 1454 images.
moe_szyslak               :: 1452 images.
lisa_simpson              :: 1354 images.
bart_simpson              :: 1342 images.
marge_simpson             :: 1291 images.
krusty_the_clown          :: 1206 images.
principal_skinner         :: 1194 images.
charles_montgomery_burns  :: 1193 images.
milhouse_van_houten       :: 1079 images.
chief_wiggum              ::  986 images.
abraham_grampa_simpson    ::  913 images.
sideshow_bob              ::  877 images.
apu_nahasapeemapetilon    ::  623 images.
kent_brockman             ::  498 images.
comic_book_guy            ::  469 images.
edna_krabappel            ::  457 images.
nelson_muntz              ::  358 images.
lenny_leonard             ::  310 images.
mayor_quimby              ::  246 images.
waylon_smithers           ::  181 images.
maggie_simpson            ::  128 images.
groundskeeper_willie      ::  121 images.
barney_gu

## Searching for more data
As we can see in the above, some characters like Homer, Ned, Moe etc. have more than 1000 images whereas Lionel Hutz and Troy McClure have less then 10.

Ideally we want to balance the number of images per class (character) somewhat and so we will first get rid of a few characters which have a low number of images. Then we will use the `google_images_download` package to search for more images for some characters.

In [10]:
character_observations_sub_dict = {}

for character, num_images in character_observations_dict.items():
  if num_images >= 500:
    character_observations_sub_dict[character] = num_images
  else:
    shutil.rmtree('/content/simpsons/simpsons_dataset/simpsons_dataset/' + character)

characters = os.listdir('/content/simpsons/simpsons_dataset/simpsons_dataset')
print('There are now ' + str(len(characters)) + ' characters.')
    
listofTuples = sorted(character_observations_sub_dict.items(), reverse=True,
                      key=lambda x: x[1])
 
# Iterate over the sorted sequence
for elem in listofTuples :
    print("{0:<26}{1:^2}{2:>13}".format(elem[0],"::", str(elem[1]) + " images."))


There are now 14 characters.
homer_simpson             :: 2246 images.
ned_flanders              :: 1454 images.
moe_szyslak               :: 1452 images.
lisa_simpson              :: 1354 images.
bart_simpson              :: 1342 images.
marge_simpson             :: 1291 images.
krusty_the_clown          :: 1206 images.
principal_skinner         :: 1194 images.
charles_montgomery_burns  :: 1193 images.
milhouse_van_houten       :: 1079 images.
chief_wiggum              ::  986 images.
abraham_grampa_simpson    ::  913 images.
sideshow_bob              ::  877 images.
apu_nahasapeemapetilon    ::  623 images.


Finding the mean of the number of images we have for each character, not including Homer.

In [13]:
i = 0
n = 0
for character, num_images in character_observations_sub_dict.items():
  if character != 'homer_simpson':
    i += num_images
    n += 1

mean = i/n
print('Mean =', mean)

Mean = 1151.076923076923


Downloading an extra 100 images (current limit of `google_images_download` without extra configuration) for all characters that have less then the mean number of images found above.



In [14]:
response = google_images_download.googleimagesdownload()   #class instantiation

for character, num_images in character_observations_sub_dict.items():
  if num_images < mean:
    character_split = character.replace("_"," ")

    arguments_content = {"keywords": character_split,
                         "limit": 100,
                         "output_directory":"/content/simpsons/simpsons_dataset/simpsons_dataset/",
                         "image_directory":character,
                         "format":"jpg"}   #creating dict of arguments
    paths = response.download(arguments_content)   #passing the arguments to the function


Item no.: 1 --> Item name = milhouse van houten
Evaluating...
Starting Download...
Completed Image ====> 1.10.jpg
Completed Image ====> 2.Milhouse_Van_Houten.jpg
Wrong image format returned. Skipping...
Completed Image ====> 3.p-the-simpsons-pamela-hayden.jpg
Wrong image format returned. Skipping...
Completed Image ====> 4.13a48fe3c8f11d7eb0f18a637e52d637.jpg
Completed Image ====> 5.20091113_millhouse_250x375.jpg
Completed Image ====> 6.maxresdefault.jpg
Completed Image ====> 7.imgbin-milhouse-van-houten-bart-simpson-homer-simpson-maggie-simpson-krusty-the-clown-bart-simpson-nQhREibR6yjGvCTXByaDAyC9F.jpg
Completed Image ====> 8.qk34NX3S_400x400.jpg
Completed Image ====> 9.maxresdefault.jpg
Completed Image ====> 10.A-1400315-1266496119.png.jpg
Completed Image ====> 11.50345603-352-k298328.jpg
Wrong image format returned. Skipping...
Wrong image format returned. Skipping...
Completed Image ====> 12.imgbin-milhouse-van-houten-bart-simpson-homer-simpson-mr-burns-mayor-quimby-bart-simpson-

Checking how many images for each character we now have.

In [15]:
characters = os.listdir('/content/simpsons/simpsons_dataset/simpsons_dataset')
for character in characters:
  num_images = len(os.listdir('/content/simpsons/simpsons_dataset/simpsons_dataset/' 
                              + character))
  character_observations_sub_dict[character] = num_images
  
listofTuples = sorted(character_observations_sub_dict.items(), reverse=True,
                      key=lambda x: x[1])
 
# Iterate over the sorted sequence
for elem in listofTuples :
    print("{0:<26}{1:^2}{2:>13}".format(elem[0],"::", str(elem[1]) + " images."))

homer_simpson             :: 2246 images.
ned_flanders              :: 1454 images.
moe_szyslak               :: 1452 images.
lisa_simpson              :: 1354 images.
bart_simpson              :: 1342 images.
marge_simpson             :: 1291 images.
krusty_the_clown          :: 1206 images.
principal_skinner         :: 1194 images.
charles_montgomery_burns  :: 1193 images.
milhouse_van_houten       :: 1161 images.
chief_wiggum              :: 1067 images.
abraham_grampa_simpson    ::  991 images.
sideshow_bob              ::  963 images.
apu_nahasapeemapetilon    ::  711 images.


Now we wish to store the images in a directory structure such that they can be used with the `ImageDataGenerator flow_from_directory`.

The below function will help us transfer data from a directory and split it into two different directories (a training and validation set) randomly and at a ratio you specify.

In [0]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
  files_list = os.listdir(SOURCE)
  files_list = random.sample(files_list, len(files_list))
  
  for file in files_list:
    file_path = SOURCE + file
    if os.path.getsize(file_path) <= 0:
      files_list.remove(file)
      print(file + " is zero length, so ignoring.") 

  n = int(SPLIT_SIZE * len(files_list))
  
  for file in files_list[0:n]:
    shutil.copyfile(SOURCE + file, TRAINING + file)
      
  for file in files_list[n:]:
    shutil.copyfile(SOURCE + file, TESTING + file)
 

In [0]:
# Creating the directory structure for ImageDataGenerator
base_dir = '/content/simpsons_generator'
os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'training')
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(train_dir)
os.mkdir(validation_dir)

characters = os.listdir('/content/simpsons/simpsons_dataset/simpsons_dataset')
split_size = .9

for character in characters:
  char_train_dir = os.path.join(train_dir, character)
  char_vald_dir = os.path.join(validation_dir, character)
  os.mkdir(char_train_dir)
  os.mkdir(char_vald_dir)
  
  source_dir = '/content/simpsons/simpsons_dataset/simpsons_dataset/' + character + '/'
  
  split_data(source_dir, char_train_dir + '/', char_vald_dir + '/', split_size)

We also wish to split the contents of the test data in /content/simpsons/kaggle_simpson_testset/kaggle_simpson_testset into a "testing" folder in our new directory.

We run the below code to check what characters we have test images for.

In [33]:
# Getting a list of images in our test directory
images_list = os.listdir('/content/simpsons/kaggle_simpson_testset/kaggle_simpson_testset/')
images_list.sort()

# Using regex to remove numbers from list of image names
def remove(list): 
    pattern = '[0-9]'
    list = [re.sub(pattern, '', i) for i in list] 
    return list
  
images_list = remove(images_list)

# Removing subscript from file name so only character name remains
images_list = [character.replace("_.jpg","") for character in images_list]

# Creating a freqency table for each character in our test set
character_freq = {} 
  
for i in images_list: 
  if i in character_freq: 
      character_freq[i] += 1
  else: 
      character_freq[i] = 1

# Printing the frequency table
listofTuples = sorted(character_freq.items(), reverse=True,
                      key=lambda x: x[1])
 
# Iterate over the sorted sequence
for elem in listofTuples :
    print("{0:<26}{1:^2}{2:>13}".format(elem[0],"::", str(elem[1]) + " images."))

apu_nahasapeemapetilon    ::   50 images.
bart_simpson              ::   50 images.
chief_wiggum              ::   50 images.
edna_krabappel            ::   50 images.
homer_simpson             ::   50 images.
kent_brockman             ::   50 images.
krusty_the_clown          ::   50 images.
lenny_leonard             ::   50 images.
lisa_simpson              ::   50 images.
marge_simpson             ::   50 images.
mayor_quimby              ::   50 images.
moe_szyslak               ::   50 images.
nelson_muntz              ::   50 images.
principal_skinner         ::   50 images.
comic_book_guy            ::   49 images.
milhouse_van_houten       ::   49 images.
ned_flanders              ::   49 images.
abraham_grampa_simpson    ::   48 images.
charles_montgomery_burns  ::   48 images.
sideshow_bob              ::   47 images.


Now we will send the test images into our test directory within /content/simpsons_generator but only if it is for a character we are using.

In [0]:
# Creating our test images directory
os.mkdir('/content/simpsons_generator/testing')
os.mkdir('/content/simpsons_generator/testing/test_folder')

In [53]:
# Getting the list of characters we have in our training directory
character_list = images_list = os.listdir('/content/simpsons_generator/training')

# Getting a list of images in our test folder
source = '/content/simpsons/kaggle_simpson_testset/kaggle_simpson_testset/'
images_list = os.listdir(source)

# Target test directory
target = '/content/simpsons_generator/testing/test_folder/'

for image in images_list:
  character = re.sub('[0-9]', '', image).replace("_.jpg","")
  if any(character == x for x in character_list):
    shutil.copyfile(source + image, target + image)

# Checking how many images we have in our new test directory    
freq_test = {} 
test_images = os.listdir('/content/simpsons_generator/testing/test_folder')

for i in test_images: 
  character = re.sub('[0-9]', '', i).replace("_.jpg","")
  if character in freq_test: 
    freq_test[character] += 1
  else: 
    freq_test[character] = 1

# Printing the frequency table
listofTuples = sorted(freq_test.items(), reverse=True,
                      key=lambda x: x[1])
 
# Iterate over the sorted sequence
for elem in listofTuples :
  print("{0:<26}{1:^2}{2:>13}".format(elem[0],"::", str(elem[1]) + " images."))

principal_skinner         ::   50 images.
apu_nahasapeemapetilon    ::   50 images.
chief_wiggum              ::   50 images.
lisa_simpson              ::   50 images.
marge_simpson             ::   50 images.
krusty_the_clown          ::   50 images.
bart_simpson              ::   50 images.
moe_szyslak               ::   50 images.
homer_simpson             ::   50 images.
ned_flanders              ::   49 images.
milhouse_van_houten       ::   49 images.
charles_montgomery_burns  ::   48 images.
abraham_grampa_simpson    ::   48 images.
sideshow_bob              ::   47 images.


# Saving to Google Drive

In [0]:
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
zipname = 'simpsons_generator_directory'

def zipfolder(foldername, target_dir):            
    zipobj = zipfile.ZipFile(foldername + '.zip', 'w', zipfile.ZIP_DEFLATED)
    rootlen = len(target_dir) + 1
    for base, dirs, files in os.walk(target_dir):
        for file in files:
            fn = os.path.join(base, file)
            zipobj.write(fn, fn[rootlen:])

zipfolder(zipname, '/content/simpsons_generator')

# 2. Create & upload a file text file.
file1 = drive.CreateFile()
file1.SetContentFile(zipname+".zip")
file1.Upload()

After opening the data locally, I looked through the images we downloaded through the google images search for each character and deleted the ones which don't have the character in it.